In [4]:
url = 'https://raw.githubusercontent.com/chiarorosa/ia_aprendizado_maquina_basico/main/ml-dataset/kaggle-basico/diabetes.csv'

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # Avaliação de Acurácia
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


In [5]:
df = pd.read_csv(url)
df.head()
X = df.drop('Outcome', axis=1)
y = df['Outcome']



In [6]:


# Dados de exemplo
df = np.random.normal(loc=0, scale=1, size=1000)

# Criar DataFrame
df = pd.DataFrame(df, columns=['Value'])

# Calcular quartis
q1 = df['Value'].quantile(0.25)
q3 = df['Value'].quantile(0.75)

# Calcular IQR
iqr = q3 - q1

# Definir limites para outliers
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Remover outliers
df_clean = df[~((df < lower_bound) | (df > upper_bound)).any(axis=1)]

# Verificar o tamanho do DataFrame após a remoção dos outliers
print("Tamanho do DataFrame após a remoção dos outliers:", df_clean.shape[0])


Tamanho do DataFrame após a remoção dos outliers: 996


In [7]:

scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)


# Dividir em dois conjuntos, treinamento e teste, usando pareto 80/20
X_train, X_test, y_train, y_test = train_test_split(
    x_scaled,
    y,
    test_size=0.2, # pareto
    random_state=42 # reproduzivel
)

In [34]:
grade = {
    'n_neighbors': range(1, 100),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p':[1,2],
    'leaf_size': range(1, 1000),
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
}
knm = RandomizedSearchCV(
    KNeighborsClassifier(),
    grade,
    cv=15,
    scoring='accuracy',
)

knm.fit(X_train, y_train)

melhor = knm.best_estimator_

y_pred = melhor.predict(X_test)

acuracia = accuracy_score(y_test, y_pred)

print(f'Acurácia: {acuracia * 100:.2f}')
print(f'Melhor modelo: {melhor}')
print(f'Parâmetros do melhor modelo: {knm.best_params_}')

guardaAcc = 0
with open('Melhor Acurracia KNeighbors.txt', 'r+', encoding='utf-8') as f:
        try:
            linha = f.readline().strip()
            if linha.startswith('Acurácia:'):
                guardaAcc = float(linha.split(': ')[1]) / 100
                print(f'Acurácia anterior: {guardaAcc * 100:.2f}')
                if acuracia > guardaAcc:
                    f.seek(0) # vai pro inicio
                    f.write(f'Acurácia: {acuracia * 100:.2f} \n')
                    
                    f.truncate() #  remove residuos antigos
                    print(f'Acurácia atualizada: {acuracia * 100:.2f}')
                else:
                    print('Acurácia não atualizada')
        except Exception as e:
            print(f"Erro ao ler o arquivo: {e}")
       


Acurácia: 74.68
Melhor modelo: KNeighborsClassifier(algorithm='brute', leaf_size=188, metric='euclidean',
                     n_neighbors=42, weights='distance')
Parâmetros do melhor modelo: {'weights': 'distance', 'p': 2, 'n_neighbors': 42, 'metric': 'euclidean', 'leaf_size': 188, 'algorithm': 'brute'}
Acurácia anterior: 79.87
Acurácia não atualizada
